In [41]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)

PM25  AQI_h AQI_h_Polutant AQI_h_label  \
site_id time                                                          
49      2016-02-05 14:00:00     0      0           PM25        Good   
        2016-02-05 15:00:00    68     68           PM25    Moderate   
        2016-02-05 16:00:00    75     75           PM25    Moderate   
        2016-02-05 17:00:00    78     78           PM25    Moderate   
        2016-02-05 18:00:00    80     80           PM25    Moderate   
...                           ...    ...            ...         ...   
        2021-02-28 20:00:00    21     21           PM25        Good   
        2021-02-28 21:00:00    24     24           PM25        Good   
        2021-02-28 22:00:00    25     25           PM25        Good   
        2021-02-28 23:00:00    24     24           PM25        Good   
        2021-03-01 00:00:00    23     23           PM25        Good   

                               NowCast  AQI_h_I  
site_id time                                     
49      2016-02-05 14:00:00   0.000000        1  
        2016-02-05 15:00:00  33.750000        2  
        2016-02-05 16:00:00  37.448980        2  
        2016-02-05 17:00:00  39.080882        2  
        2016-02-05 18:00:00  39.913255        2  
...                                ...      ...  
        2021-02-28 20:00:00  10.572388        1  
        2021-02-28 21:00:00  11.785400        1  
        2021-02-28 22:00:00  12.391846        1  
        2021-02-28 23:00:00  12.195068        1  
        2021-03-01 00:00:00  11.596436        1  

[44411 rows x 6 columns]

In [ ]:
import pandas as pd
import numpy as np
from features import calculate_AQI

In [37]:
# We need to know the number of continuous values
# So when we use chunker, we can refer to the starting point to know how many to skip if timeline is not continous after skipping
# Basic idea: Loop through the dataframe, if it is continuous increase the continous counter and pos counter
# If encounter uncontinous timeline (Encounter a nan value), find the lastpoint with a loop,
# Then change the start position continous length column values to the continous counter.
# Reset the continous counter and start counter
# Result: at the start of each continous timeline, there will be a number which indicate how many step you need to skip if
# you need to get to the next timeline

def time_continous_marker(df):
    df_copy = df.copy()
    pos = 0
    start_pos = 0
    continous = 0
    size = 24
    length = df_copy.shape[0]
    while pos < length:
        try:
            if df_copy.iloc[pos : pos + size, 1].isnull().sum() == 0:
                while df_copy.iloc[pos : pos + size, 1].isnull().sum() == 0:
                    pos += size
                    continous += size
            else:
                while np.isnan(df_copy.iloc[pos, 1]) == False:
                    continous += 1
                    pos += 1
                df_copy.iloc[start_pos, 7] = continous
                continous = 0
                while np.isnan(df_copy.iloc[pos, 1]) == True:
                    pos += 1
                start_pos = pos
#             print(pos)
        except IndexError:
            df_copy.iloc[start_pos, 7] = continous
            continous = 0
            print("Current position is: {}".format(pos))
            pos = length
    return df_copy

In [ ]:
# TP HCM
import pandas as pd
import numpy as np
import glob
import xarray as xr

idx = pd.IndexSlice

_HCM_us_embass_raw_data_path = os.path.join(PROJ_ROOT,
                             "data",
                             "external",
                             "us_embass",
                             "raw")

_HCM_us_embass_data_files = glob.glob(_HCM_us_embass_raw_data_path + '/HoChiMinhCity_*YTD*.csv')
USEm_HCM_df = pd.DataFrame()
for file in _HCM_daisuquanmy_data_files:
    USEm_HCM_df = USEm_HCM_df.append(pd.read_csv(file, parse_dates=True))

# Remove site year 2015 because inconsistency data
# daisuquanmy_df = daisuquanmy_df[(daisuquanmy_df['Year'] != 2015)]
USEm_HCM_df.drop(columns=['Site', 'Parameter', 'AQI', 'AQI Category','NowCast Conc.', 'Conc. Unit', 'Duration'], inplace=True)
USEm_HCM_df['Date (LT)'] = pd.to_datetime(USEm_HCM_df['Date (LT)'])
USEm_HCM_df = USEm_HCM_df.rename(columns={'Date (LT)': 'time'})
USEm_HCM_df['site_id'] = 49
# Remove duplicate, fill in missing index and set ['Date (LT)'] as index and sort
USEm_HCM_df.drop_duplicates(subset ="time", 
                     keep = "first", inplace = True)
USEm_HCM_df = USEm_HCM_df.sort_values(by=['time'])
USEm_HCM_df = USEm_HCM_df.set_index('time').asfreq('H').sort_index()
USEm_HCM_df.reset_index(drop=False, inplace=True)
USEm_HCM_df = USEm_HCM_df.set_index(['site_id', 'time'])
# For all Raw Conc <= 0 QC Name must be Missing. Raw Conc change to -999
USEm_HCM_df.loc[USEm_HCM_df['Raw Conc.'] <= 0, 'Raw Conc.'] = -999
USEm_HCM_df.loc[USEm_HCM_df['Raw Conc.'] >= 600, 'Raw Conc.'] = -999
USEm_HCM_df.loc[USEm_HCM_df['Raw Conc.'] <= 0, 'QC Name'] = 'Missing'

USEm_HCM_df = USEm_HCM_df.replace(-999, np.nan)
# Change Raw Conc to PM25
USEm_HCM_df.rename(columns={'Raw Conc.': 'PM25'}, inplace=True)
# Fill missing data
USEm_HCM_df.fillna(method='ffill', limit=2, inplace=True)
USEm_HCM_df.fillna(method='bfill', limit=2, inplace=True)

# For year 2015 only, take data after 2015-12-09 14:00:00
USEm_HCM_df = USEm_HCM_df.loc[USEm_HCM_df.index.get_level_values(1) >= pd.to_datetime("2016-02-05 02:00 PM")]
USEm_HCM_df = USEm_HCM_df[['PM25']]

USEm_HCM_df['PM25'] = USEm_HCM_df['PM25'].interpolate()

USEm_HCM_df

In [38]:
AQI = calculate_AQI.calculate_AQI_h(USEm_HCM_df)
AQI = AQI.replace(0, np.nan).copy()
AQI = AQI[1:]
AQI['Continous length'] = 0
AQI = AQI.append(pd.Series(),ignore_index=True)
AQI = time_continous_marker(AQI)
AQI = AQI.dropna(subset=['AQI_h'])

<ipython-input-38-b1c3970015b7>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  AQI = AQI.append(pd.Series(),ignore_index=True)


Current position is: 44411


In [ ]:
_HCM_us_embass_processed_data_path = os.path.join(PROJ_ROOT,
                             "data",
                             "external",
                             "us_embass",
                             "processed")
AQI.to_csv(_HCM_us_embass_processed_data_path + "/processed_hcm.csv")

In [ ]:
# Now chunker need to be able to output chunks of train data and target in correct order
# Basic ideas: loop through each chunk of data (size is size with ,
# if the timeline is continous, add data to chunk_data [pos: pos+size]
# add data to target [pos+size+target_hour]
# if the timeline is not, skip pos according to size
# Repeat below till timeline is continous
# if after skipping, the timeline is still not continous, skip pos according to continous length colunmns
from IPython.core.debugger import set_trace
def chunker(seq, size, target_hour, debug = True):
    """
    Input: 
        + Dataframe with PM25, AQI_h, AQI_h_I, Continous length and some other columns
            - If data doesn't have Continous length columns, add 1 to data
            - Take only PM25, date columns as train
            - Take only AQI_h, AQI_h_I as target
        + size: length of each chunk
        + target_hour: labels for the hour which will be predicted
    Ouput: 2 lists: chunk_data, target
        + chunk_data: original data chunked to a specific timestep, have shape [..., timeframe, features]
        + target: Label for each chunked train/test data, have shape [..., 1]
    """
    if not 'Continous length' in seq.columns:
        seq['Continous length'] = 0

    timerange = pd.Timedelta(hours=size + target_hour)
    chunk_data = []
    target = []
    multiclass_target = []
    pos = 0
    length = seq.shape[0]
    while pos < (length - size):
        try:
            if seq.iloc[pos + size + target_hour].name[1] - seq.iloc[pos].name[1] == timerange:
                while seq.iloc[pos + size + target_hour].name[1] - seq.iloc[pos].name[1] == timerange:
                    chunk_data.append(seq.iloc[pos:pos + size].loc['PM25','Hour','Day of Week'])
                    target.append(seq.iloc[pos + size + target_hour].loc['AQI_h'])
                    multiclass_target.append(seq.iloc[pos + size + target_hour].loc['AQI_h_I'])
                    pos += 1
                    if pos + size + target_hour >= length - size:
                        print("Returned")
                        return np.array(chunk_data), np.array(target), np.array(multiclass_target)
            else:
                tmp_pos = pos + size + target_hour
                while seq.iloc[tmp_pos + size + target_hour].name[1] - seq.iloc[tmp_pos].name[1] != timerange:
                    tmp_pos += int(seq.iloc[tmp_pos, 6])
                print("Jump from {} to {}".format(pos, tmp_pos))
                pos = tmp_pos
        except IndexError:
            print("Current position is: {}".format(pos))
            print("Current tmp position is: {}".format(tmp_pos))
            pos = length - size
    print("Returned")
    return np.array(chunk_data), np.array(target), np.array(multiclass_target)
chunk_data, target, multiclass_target = chunker(AQI_copy, 12, 0)